In [ ]:
%load_ext autoreload
%autoreload 2
from env_utils import get_4wt_symmetric_env, get_lhs_env
from wind_processes import SetSequenceWindProcess
import numpy as np
from PIL import Image
import time
from tqdm import tqdm
from dynamic_windfarm_env import DynamicPriviegedWindFarmEnv


# turbine_layout = [[0, 0, 250, 250], [1, 1, 251, 251]]

# turbine_layout = [[600, 1500, 2400, 600, 1500, 2400, 600, 1500, 2400], [600, 600, 600, 1500, 1500, 1500, 2400, 2400, 2400]]
# mast_layout = [[0, 0, 3000, 3000], [0, 3000, 0, 3000]]
# wind_directions = np.concatenate((
#     np.ones(300//5) * 255,
#     np.linspace(255, 195, 300//5),
#     np.ones(300//5) * 195,
# ))
# update_rule = None 
update_rule = 'observation_points'

turbine_layout = [[400, 1300], [500, 500]]
mast_layout = [[0, 0, 2000, 2000], [0, 2000, 0, 2000]]
wind_directions = np.ones(1000//5) * 270

EPISODES = 1
EPISODE_LENGTH = len(wind_directions)


if len(turbine_layout) == 2:
    time_delta = 1
    op_per_turbine = 3
    op_wake_matrix_horizon = 20
elif len(turbine_layout) == 9:
    time_delta = 1
    op_per_turbine = 3
    op_wake_matrix_horizon = 200

env = DynamicPriviegedWindFarmEnv(
    turbine_layout=turbine_layout,
    #lidar_observations=('wind_speed', 'wind_direction'),
    mast_layout=mast_layout,
    floris="myfloris.json",
    episode_length=EPISODE_LENGTH,
    sorted_wind=False,
    wind_speed=None,
    changing_wind=False,
    action_representation='wind',
    observation_noise=0.0,
    verbose=False,
    update_rule=update_rule,
    load_pyglet_visualization=True,
    parallel_dynamic_computations=False,
    time_delta=time_delta,
    op_per_turbine=op_per_turbine,
    op_wake_matrix_horizon=op_wake_matrix_horizon,
)
env.op_per_turbine = 5
env.time_delta = 5
env.op_wake_matrix_horizon = 200
# env = get_4wt_symmetric_env(
#     load_pyglet_visualization=False, privileged=True, 
#     sorted_wind=True, wind_speed=8.0, dynamic_mode='observation_points', verbose=False)
# env = get_lhs_env("lhs_env_nt8_md150_wb750x750",#"lhs_env_nt16_md75_wb1500x1500", 
#     mast_distancing=150, load_pyglet_visualization=True, 
#     verbose=False,)
# env = get_lhs_env("lhs_env_nt16_md75_wb1500x1500", 
#     mast_distancing=150, load_pyglet_visualization=True, 
#     verbose=False,)
env.seed(0)


# Give it a nice wind process
# Concatenate 2 linspaces
env.save_double_plot = True
# wind_directions = np.concatenate((np.ones((3))*270, np.linspace(270, 250, 20), np.linspace(250, 290, 20)))
# wind_directions = np.linspace(255, 195, 100)

#wind_directions = np.arange(0, 359, 360/len(wind_directions))
wind_speeds = np.ones_like(wind_directions) * 8.0
wind_process = SetSequenceWindProcess(wind_speeds, wind_directions)
env.wind_process = wind_process

# Give it very obvious momentum
# env.momentum_alpha = 1
# env.momentum_beta = 1

env.reset()

durations = []
for i in range(EPISODES):
    for j in tqdm(range(EPISODE_LENGTH)):
        t0 = time.time()
        action = env.action_space.sample()*0
        obs, reward, t, tr, info = env.step(action)
        # if j in [60, 90, 120]:
        #     print(f"Step {j}")
        env.render()
        duration = time.time() - t0
        durations.append(duration/env.time_delta)
    # env.render()
    env.reset()
env.close()

print(f"Average episode duration: {np.mean(durations)}")
print(f"Average episode duration std: {np.std(durations)}")

# Plot the step and render times
import matplotlib.pyplot as plt
plt.plot(duration)
plt.show()

In [ ]:
# make a gif out of the pngs
from PIL import Image

runs = [
    # "floris9", 
    # "floris2", 
    "dynamic9", 
    "dynamic2",
]
lens = [180, 200, 180, 200]
for run, length in zip(runs, lens):
    image_paths = [f"figures/renders/{run}/step_{i+1}.png" for i in range(length)]
    images = [Image.open(x) for x in image_paths]
    images[0].save(f'figures/renders/{run}_episode.gif',
                save_all=True, append_images=images[1:], optimize=False, duration=50, loop=0)
    print(f"Gif saved for {run}")
print("Gif saved")

In [ ]:
print(f"Average episode duration: {np.mean(durations)}")
print(f"Average episode duration std: {np.std(durations)}")

In [ ]:
# Plot the step and render times
import matplotlib.pyplot as plt
plt.plot(step_times, label="Step")
plt.plot(render_times, label="Render")
plt.legend()
plt.show()
print("Mean step time: ", np.mean(step_times), 1/np.mean(step_times))
print("To do 1000 steps of delta 1: ", 1000*np.mean(step_times))
print("To do 1 set of 1000s eddy simulation: ", 44*60*60)
print("Ratio: ", 44*60*60/(1000*np.mean(step_times)))

In [ ]:
import numpy as np

a = np.arange(0, 10)
print(a)
# repeat in 2D
b = np.tile(a, (10, 1))
print(b)
print(b[0])

In [ ]:
import numpy as np

np.sum([10, np.nan, 20])


# Plot momentum environments

This relies on a temp change where reset did not reset the current_plot variable,
so that sorted wind could be used to plot with sorted wind directions (should use a different
wind process now..., but it did the job of rendering the env)

In [ ]:
alpha = [0.5, 0.75, 0.9, 0.95, 0.99, 1]
beta = [0.5, 0.75, 0.9, 0.95, 0.99, 1]

env = get_4wt_symmetric_env(
    load_pyglet_visualization=True, privileged=True, 
    sorted_wind=True, wind_speed=8.0, update_rule='momentum')
# Give it a nice wind process
wind_directions = np.linspace(250, 290, 40, endpoint=False)
wind_speeds = np.ones_like(wind_directions) * 8.0
wind_process = SetSequenceWindProcess(wind_speeds, wind_directions)
env.wind_process = wind_process

# iterate over all combinations
for a in alpha:
    for b in beta:
        imgs = []
        env.reset()
        env.update_rule = 'momentum'
        env.momentum_alpha = a
        env.momentum_beta = b
        print(f'alpha: {a}, beta: {b}')
        for i in range(40):
            action = env.action_space.sample()
            obs, reward, t, tr, info = env.step(action)
            img = env.render(mode='rgb_array')
            imgs.append(img)
        imgs_pil = [Image.fromarray(img) for img in imgs]
        #assert False, 'This will overwrite the files'
        imgs_pil[0].save(f"figures/renders/momentum/alpha_{a}_beta_{b}.gif", save_all=True, append_images=imgs_pil[1:], loop=0, duration=100)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

data = pd.DataFrame(columns=["Wind", "Simulator", "Speed"])
fixed_wind = "Fixed, 2 turbines"
changing_wind = "Changing, 3x3 turbines"
data.loc[len(data)] = [fixed_wind, "SOWFA", 0.29e4]
data.loc[len(data)] = [changing_wind, "SOWFA", 0.27e5]
data.loc[len(data)] = [fixed_wind, "Floris gym", 0.34e-2]
data.loc[len(data)] = [changing_wind, "Floris gym", 0.12e-1]
data.loc[len(data)] = [fixed_wind, "Quasi-Dynamic", 0.7e-1]
data.loc[len(data)] = [changing_wind, "Quasi-Dynamic", 0.41]

fig, ax = plt.subplots()
sns.barplot(data=data, x="Wind", y="Speed", hue="Simulator", ax=ax)
# log plot
ax.set_yscale('log')
ax.set_ylabel("Simulation speed (compute time / simulation time)")
ax.set_xlabel("Wind scenario")
plt.title("Simulation speed comparison")
plt.show()